# Constrain plotted coins by market cap and volume
## Click on the graph to take you to a coin's page on coinmarketcap.com!

In [1]:
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import time
import webbrowser

%matplotlib auto

Using matplotlib backend: Qt5Agg


In [2]:
coinListGet = requests.get('https://coinmarketcap.com/all/views/all/')
coinListSoup = BeautifulSoup(coinListGet._content, 'html.parser')

data = {}
for num, row in enumerate(coinListSoup.findAll('tr')):
    if num >= 1:
        try:
            name = row.findChild('a', {'class': 'currency-name-container'}).get_text()
            symbol = row.findChild('span', {'class': 'currency-symbol'}).get_text()
            href = row.findChild('a')['href']
            price = float(row.findChild('a', {'class': 'price'}).get_text().strip('$').replace(',', ''))
            volume = float(row.findChild('a', {'class': 'volume'}).get_text().strip('$').replace(',', ''))
            marketcap = float(row.findChild('td', {'class' : 'market-cap'}).get_text().strip().strip('$').replace(',', ''))
        
            data[symbol] = {
                'name' : name,
                'symbol': symbol,
                'href': href,
                'price': price,
                'volume': volume,
                'marketcap': marketcap
            }
        except ValueError as err:
            # Pops up because of low volumes
            continue

summary = pd.DataFrame.from_dict(data, orient='index').reset_index()

In [3]:
# View the whole market
fig, mcap_ax = plt.subplots(figsize=(12, 9))
vol_ax = mcap_ax.twinx()
summary.sort_values(by='marketcap').plot(x=summary.index, y='marketcap', marker='o', linestyle='', ax=mcap_ax, color='C0', logy=True)
summary.sort_values(by='marketcap').plot(x=summary.index, y='volume', marker='o', linestyle='', ax=vol_ax, color='C2', logy=True)
mcap_ax.legend(loc=2)
vol_ax.legend(loc=1)

mcap_ax.tick_params(axis='x', which='both', bottom='off', top='off', labelbottom='off')
mcap_ax.set_ylabel('Market Cap ($)')
mcap_ax.set_xlabel('Coins ranked by market cap')
vol_ax.set_ylabel('24 Hr Volume ($)')

fig.tight_layout()

In [4]:
# After viewing the whole market, constrain the coins by marketcap, volume, or price
minCap = (summary['marketcap'] > 5e5)
maxCap = (summary['marketcap'] < 3e6)
minVol = (summary['volume'] > 1e4)
constrained = summary[minCap & maxCap & minVol]
constrained = constrained.sort_values(by='marketcap').reset_index()

fig, (mcap_ax, perc_ax) = plt.subplots(nrows=2, figsize=(12, 9))
vol_ax = mcap_ax.twinx()
constrained.plot(y='marketcap', marker='o', linestyle='', ax=mcap_ax, color='C0', logy=True)
constrained.plot(y='volume', marker='o', linestyle='', ax=vol_ax, color='C2', logy=False)
perc_ax.plot(constrained['volume'] / constrained['marketcap'], 'o')

mcap_ax.legend(loc=2)
mcap_ax.tick_params(axis='x', which='both', bottom='off', top='off', labelbottom='off')
mcap_ax.set_ylabel('Market Cap ($)')

vol_ax.legend(loc=1)
vol_ax.set_ylabel('24 Hr Volume ($)')

perc_ax.tick_params(axis='x', which='both', bottom='off', top='off', labelbottom='off')
perc_ax.set_yticklabels(['{:3.2f}%'.format(x*100) for x in perc_ax.get_yticks()])
perc_ax.set_xlabel('Coins ranked by market cap')
perc_ax.set_ylabel('Volume / Market Cap (%)')


def onclick(event):
    pos = int(np.round(event.xdata))
    webbrowser.open(f"https://coinmarketcap.com{constrained.loc[pos, 'href']}")

fig.canvas.mpl_connect('button_press_event', onclick)
fig.tight_layout()